# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [19]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [20]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [21]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [22]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [23]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [24]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [25]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [26]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [27]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [28]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [29]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [30]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [31]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

2024-05-24 06:42:33,525 - 139633211546496 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [32]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [33]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {
    "coworker": "Data Analyst",
    "task": "Continuously monitor and analyze market data for AAPL to identify trends and predict market movements",
    "context": "The task involves using statistical modeling and machine learning to analyze market data for the selected stock AAPL. The goal is to identify trends and predict market movements to provide insights and alerts about significant market opportunities or threats for AAPL."
}


> Entering new CrewAgentExecutor chain...
I need to gather real-time mark

Final Answer: Here is the complete content for AAPL stock on Yahoo Finance:

Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
 News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science 2024 election Originals The 360 Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Shopping Finance My Portfolio My Portfolio News Latest News Stock Market Originals Premium News Economics Housing Earnings Tech Crypto Biden Economy EV Deep Dive Markets Stocks: Most Actives Stocks: Gainers Stocks: Losers Calendars Trending Tickers Futures World Indices US Treasury Bonds Currencies Crypto Top ETFs Top Mutual Funds Highest Open I

Final Answer: Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
 News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science 2024 election Originals The 360 Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Shopping Finance My Portfolio My Portfolio News Latest News Stock Market Originals Premium News Economics Housing Earnings Tech Crypto Biden Economy EV Deep Dive Markets Stocks: Most Actives Stocks: Gainers Stocks: Losers Calendars Trending Tickers Futures World Indices US Treasury Bonds Currencies Crypto Top ETFs Top Mutual Funds Highest Open Interest Highest Implied Volatility Stock Comparison Advanced Ch

I need to delegate tasks to the appropriate team members and ask the right questions to ensure the trading strategies align with the user's risk tolerance.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Provide insights on AAPL stock for potential trading strategies", "context": "I need detailed analysis on Apple Inc. (AAPL) stock price movements, trends, and any relevant news that could impact trading strategies."}

> Entering new CrewAgentExecutor chain...
I need to gather detailed information on AAPL stock price movements, trends, and any relevant news that could impact trading strategies. I should use the tools available to gather this data.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/quote/AAPL?p=AAPL"} 

Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
 News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate chang

Final Answer: 
Apple Inc. (AAPL) stock closed at $186.88 on May 23, with a decrease of 2.11%. It has shown a negative trend in the short term but a positive trend over the long term, with a 5-year return of 308.97%. The company operates in the consumer electronics and technology sector, offering a range of products and services. Its market cap stands at 2.866T, and it has demonstrated solid financial performance with a profit margin of 26.31% and a return on equity of 147.25%. Overall, AAPL has been a strong performer in the market, and its long-term performance indicates potential for growth.

> Finished chain.
 

Apple Inc. (AAPL) stock closed at $186.88 on May 23, with a decrease of 2.11%. It has shown a negative trend in the short term but a positive trend over the long term, with a 5-year return of 308.97%. The company operates in the consumer electronics and technology sector, offering a range of products and services. Its market cap stands at 2.866T, and it has demonstrated soli

I need to delegate tasks to my team members and ask specific questions to gather all the necessary information for the best execution plans for AAPL.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Analyze approved trading strategies for AAPL", "context": "I need detailed insights on the approved trading strategies for AAPL, considering current market conditions and optimal pricing."}

> Entering new CrewAgentExecutor chain...
I should start by reading the website content to gather information on approved trading strategies for AAPL.

Action: Read website content
Action Input: {"website_url": "https://www.investopedia.com/articles/trading/09/trading-strategies-apple-aapl-stock.asp"} 

Page Not Found
By clicking “Accept All Cookies”, you agree to the storing of cookies on your device to enhance site navigation, analyze site usage, and assist in our marketing efforts. Cookies Settings Accept All Cookies
 Investing
 Stocks
 Cryptocurrency
 Bonds
 ET

I need to ensure that the risks associated with the proposed trading strategies and execution plans for AAPL are thoroughly evaluated and mitigated to the best of my ability. I will need to delegate tasks to my team members and ask specific questions to gather all necessary information for a comprehensive risk analysis report.

Action: Delegate work to co-worker
Action Input: {"coworker": "Risk Advisor", "task": "Conduct a detailed risk analysis for the proposed trading strategies and execution plans for AAPL", "context": "I have compiled detailed execution plans for AAPL, including short-term, long-term, and diversification strategies to capitalize on the stock's performance and manage risk."}

> Entering new CrewAgentExecutor chain...
I need to gather as much information as possible to conduct a thorough risk analysis for the proposed trading strategies and execution plans for AAPL.

Action: Read website content
Action Input: {"website_url": "https://www.apple.com/investor/"} 

Atten

Final Answer: 

Risk Analysis Report for AAPL:

Short-Term Strategy Risks:
1. Intraday Price Volatility: The short-term strategy relies on closely monitoring intraday price movements. Sudden fluctuations in price can lead to unexpected losses if stop-loss orders are not executed correctly.
2. Market Sentiment: Negative market sentiment towards AAPL can impact the success of the short-term strategy, especially if there are external factors affecting the stock price.
3. Execution Timing: Timing the execution of trades according to the short-term strategy can be challenging due to the fast-paced nature of intraday trading.

Mitigation Strategies:
1. Use advanced trading tools and platforms to set up automated stop-loss orders and alerts to manage price volatility effectively.
2. Stay informed about market news and sentiment analysis to anticipate any potential shifts in market sentiment towards AAPL.
3. Develop a clear execution plan with specific entry and exit points to minimize the ris

- Display the final result as Markdown.

In [34]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL:

Short-Term Strategy Risks:
1. Intraday Price Volatility: The short-term strategy relies on closely monitoring intraday price movements. Sudden fluctuations in price can lead to unexpected losses if stop-loss orders are not executed correctly.
2. Market Sentiment: Negative market sentiment towards AAPL can impact the success of the short-term strategy, especially if there are external factors affecting the stock price.
3. Execution Timing: Timing the execution of trades according to the short-term strategy can be challenging due to the fast-paced nature of intraday trading.

Mitigation Strategies:
1. Use advanced trading tools and platforms to set up automated stop-loss orders and alerts to manage price volatility effectively.
2. Stay informed about market news and sentiment analysis to anticipate any potential shifts in market sentiment towards AAPL.
3. Develop a clear execution plan with specific entry and exit points to minimize the risks associated with timing trades.

Long-Term Strategy Risks:
1. Market Fluctuations: Long-term holding of AAPL shares exposes the portfolio to market fluctuations over an extended period. Economic downturns or industry-specific challenges can impact the stock price.
2. Competitive Landscape: Increasing competition in the tech industry can affect AAPL's market share and long-term growth potential.
3. Regulatory Changes: Changes in regulations or legal challenges can impact AAPL's operations and stock performance in the long run.

Mitigation Strategies:
1. Diversify the investment portfolio to include other stable assets alongside AAPL shares to reduce the impact of market fluctuations.
2. Conduct thorough research on AAPL's competitive position and future growth prospects to make informed decisions on long-term holdings.
3. Stay updated on regulatory developments and legal challenges facing AAPL to anticipate potential risks and adjust the investment strategy accordingly.

Diversification Strategy Risks:
1. Correlation Risk: Investing in other tech stocks or ETFs alongside AAPL may expose the portfolio to correlated risks, especially during market downturns.
2. Sector-Specific Risks: The broader technology sector may face industry-specific challenges that can impact the performance of the diversified portfolio.
3. Market Volatility: Rapid changes in market conditions can affect the performance of tech stocks and ETFs, leading to higher volatility in the portfolio.

Mitigation Strategies:
1. Maintain a balanced portfolio with a mix of assets across different sectors to reduce correlation risk and enhance diversification benefits.
2. Conduct regular reviews of the performance of tech stocks and ETFs in the portfolio to identify any sector-specific risks and adjust the allocation accordingly.
3. Implement risk management strategies such as setting stop-loss orders and portfolio rebalancing to mitigate the impact of market volatility on the diversified portfolio.

Overall, the risk analysis report highlights the key risks associated with the proposed trading strategies and execution plans for AAPL, along with detailed mitigation strategies to manage these risks effectively. By considering the potential risks and implementing appropriate risk management measures, the investment strategy can align with the user's preferences and financial goals while maximizing returns and minimizing losses.